<div class="alert alert-block alert-info">



Business Case 1<a class="anchor"><a id='toc'></a></b><br>
* [<font color='#E8800A'>Import Libraries</font>](#first-bullet)<br>
- [<font color='#E8800A'>Extracting Time Series Data</font>](#second-bullet)<br>
- [<font color='#E8800A'>Data Exploration</font>](#third-bullet)<br>
- [<font color='#E8800A'>Missing Cases</font>](#fourth-bullet)<br>
- [<font color='#E8800A'>Encoding Categorical Features</font>](#fifth-bullet)<br>
- [<font color='#E8800A'>Data Partition</font>](#sixth-bullet)<br>
- [<font color='#E8800A'>Outliers</font>](#seventh-bullet)<br>
- [<font color='#E8800A'>Normalization</font>](#eighth-bullet)<br>
- [<font color='#E8800A'>Classification</font>](#nineth-bullet)<br>
- [<font color='#E8800A'>Features Selection</font>](#tenth-bullet)<br>
- [<font color='#E8800A'>Ballancing Dataset</font>](#eleventh-bullet)<br>
- [<font color='#E8800A'>Tuning Models</font>](#twelveth-bullet)<br>
- [<font color='#E8800A'>Models Performance Comparison</font>](#thirteenth-bullet)<br>
- [<font color='#E8800A'>Testing the Model</font>](#fourteenth-bullet)<br>
- [<font color='#E8800A'>Lift Curve</font>](#fifteenth-bullet)<br>

</div>



# <font color='#E8800A'>Import Libraries</font> <a class="anchor" id="first-bullet"></a>
[Back to TOC](#toc)

In [29]:
import numpy as np
import scipy
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.base import clone

# imports for visualizations
import matplotlib.pyplot as plt
import seaborn as sns
from math import ceil
from itertools import product
from pandas_profiling import ProfileReport
%matplotlib inline
%config InlineBackend.figure_format = 'retina' 
import matplotlib.cm as cm
sns.set()

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.neighbors import LocalOutlierFactor
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, classification_report, precision_recall_curve
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from imblearn.over_sampling import SMOTENC
from collections import Counter
from scikitplot.metrics import plot_cumulative_gain, plot_lift_curve
from sklearn.model_selection import RandomizedSearchCV

In [30]:
df_minmax_X = pd.read_csv("H2_preprocessdone.csv").drop(['Unnamed: 0'], axis=1)
df_minmax_X

IsCanceled  LeadTime  ArrivalDateWeekNumber  ArrivalDateDayOfMonth  \
0               0  0.009539                     27               0.000000   
1               1  0.139905                     27               0.000000   
2               1  0.103339                     27               0.000000   
3               1  0.146264                     27               0.000000   
4               1  0.158983                     27               0.033333   
...           ...       ...                    ...                    ...   
79325           0  0.036566                     35               0.966667   
79326           0  0.162162                     35               1.000000   
79327           0  0.054054                     35               1.000000   
79328           0  0.173291                     35               1.000000   
79329           0  0.325914                     35               0.933333   

       StaysInWeekendNights  StaysInWeekNights  Adults  Children  Babies  \
0                     0.000           0.048780    0.25       0.0     0.0   
1                     0.000           0.097561    0.50       0.0     0.0   
2                     0.000           0.097561    0.25       0.0     0.0   
3                     0.125           0.097561    0.50       0.0     0.0   
4                     0.000           0.048780    0.50       0.0     0.0   
...                     ...                ...     ...       ...     ...   
79325                 0.125           0.121951    0.50       0.0     0.0   
79326                 0.125           0.121951    0.75       0.0     0.0   
79327                 0.125           0.121951    0.50       0.0     0.0   
79328                 0.125           0.121951    0.50       0.0     0.0   
79329                 0.125           0.170732    0.50       0.0     0.0   

       PreviousCancellations  ...  x5_HB         x5_SC         x6_1  \
0                        0.0  ...           1.0           0.0   0.0   
1                        0.0  ...           0.0           0.0   0.0   
2                        0.0  ...           0.0           0.0   0.0   
3                        0.0  ...           0.0           0.0   0.0   
4                        0.0  ...           0.0           0.0   0.0   
...                      ...  ...           ...           ...   ...   
79325                    0.0  ...           0.0           0.0   0.0   
79326                    0.0  ...           0.0           0.0   0.0   
79327                    0.0  ...           0.0           0.0   0.0   
79328                    0.0  ...           0.0           0.0   0.0   
79329                    0.0  ...           1.0           0.0   0.0   

       x7_B                 x7_C                 x7_D                 \
0                      0.0                  0.0                  0.0   
1                      0.0                  0.0                  0.0   
2                      0.0                  0.0                  0.0   
3                      0.0                  0.0                  0.0   
4                      0.0                  0.0                  0.0   
...                    ...                  ...                  ...   
79325                  0.0                  0.0                  0.0   
79326                  0.0                  0.0                  0.0   
79327                  0.0                  0.0                  1.0   
79328                  0.0                  0.0                  0.0   
79329                  0.0                  0.0                  0.0   

       x7_E                 x7_F                 x7_G                 \
0                      0.0                  0.0                  0.0   
1                      0.0                  0.0                  0.0   
2                      0.0                  0.0                  0.0   
3                      0.0                  0.0                  0.0   
4                      0.0                  0.0                  0.0   
...                

## SPLIT TRAIN_TEST

In [31]:
target = ["IsCanceled"]
features= df_minmax_X.columns.drop(target).to_list()
data =df_minmax_X[features]
target= df_minmax_X[target]

In [32]:
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    test_size=0.2, 
                                                    random_state=15, 
                                                    shuffle=True, 
                                                    stratify=target
                                                   )


In [33]:
X_train, X_val, y_train, y_val = train_test_split(X_train,
                                                  y_train,
                                                  test_size = 0.25,
                                                  random_state = 15,
                                                  shuffle=True,
                                                  stratify=y_train
)

# <font color='#E8800A'>Classification</font> <a class="anchor" id="nineth-bullet"></a>
[Back to TOC](#toc)

## Create a function that will print the results of the classification report

In [34]:
model = list()
resample = list()
F1score_0 = list()
F1score_1 = list()
F1score_total = list()
precision = list()
AUCROC = list()

In [35]:
def test_eval(Clf_model, X_test, y_test, algo=None, sampling=None):
    # Test set prediction
    y_prob = Clf_model.predict_proba(X_test)
    y_pred = Clf_model.predict(X_test)

    print('Confusion Matrix')
    print('='*60)
    print(confusion_matrix(y_test,y_pred),"\n")
    print('Classification Report')
    print('='*60)
    print(classification_report(y_test,y_pred),"\n") #, labels=[0,1]
    print('AUC-ROC')
    print('='*60)
    print(roc_auc_score(y_test, y_prob[:,1]))
          
    model.append(algo)
    resample.append(sampling)
    F1score_0.append(f1_score(y_test, y_pred, average='micro', labels=[0]))
    F1score_1.append(f1_score(y_test, y_pred, average='micro', labels=[1]))
    F1score_total.append(f1_score(y_test,y_pred, average='micro'))
    precision.append(precision_score(y_test,y_pred))
    AUCROC.append(roc_auc_score(y_test, y_prob[:,1]))

### 1) Apply the Random Forest Classifier

In [36]:
#Random Forest Classifier
RF0 = RandomForestClassifier()
RF0.fit(X_train, y_train)
y_pred_RF = RF0.predict(X_val)
f1_score_RF = f1_score(y_val, y_pred_RF, average='micro')
print('The f1_score of the Random Forest is:', f1_score_RF)
test_eval(RF0, X_val, y_val, 'Random Forest', sampling=None)


The f1_score of the Random Forest is: 0.8637967981847977
Confusion Matrix
[[8530  716]
 [1445 5175]] 

Classification Report
              precision    recall  f1-score   support

           0       0.86      0.92      0.89      9246
           1       0.88      0.78      0.83      6620

    accuracy                           0.86     15866
   macro avg       0.87      0.85      0.86     15866
weighted avg       0.86      0.86      0.86     15866
 

AUC-ROC
0.9385302078860916


## 2) Apply the Gradient Boost Classifier

In [37]:
GB0 = GradientBoostingClassifier(random_state = 5)
GB0.fit(X_train, y_train)
y_pred_GB = GB0.predict(X_val)
f1_score_GB = f1_score(y_val, y_pred_GB, average='micro')
print('The f1_score of the Gradient Boosting is:', f1_score_GB)
test_eval(GB0, X_val, y_val, "Gradient Boosting", sampling=None)

The f1_score of the Gradient Boosting is: 0.8228286902811043
Confusion Matrix
[[8274  972]
 [1839 4781]] 

Classification Report
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      9246
           1       0.83      0.72      0.77      6620

    accuracy                           0.82     15866
   macro avg       0.82      0.81      0.81     15866
weighted avg       0.82      0.82      0.82     15866
 

AUC-ROC
0.9066276230825382


# <font color='#E8800A'>Feature Selection</font> <a class="anchor" id="tenth-bullet"></a>
[Back to TOC](#toc)

## Feature Importance GB


In [38]:
importances_GB = pd.DataFrame({'feature':X_val.columns,'importance':np.round(GB0.feature_importances_,3)})
importances_GB = importances_GB.sort_values('importance',ascending=False)


## Feature Importance RF

In [39]:
importances_RF = pd.DataFrame({'feature':X_val.columns,'importance':np.round(RF0.feature_importances_,3)})
importances_RF = importances_RF.sort_values('importance',ascending=False)


In [40]:
features_selected_RF = importances_RF[(importances_RF['importance']> 0.002)]

In [41]:
features_selected_GB = importances_GB[(importances_GB['importance']> 0.002)]

In [42]:
features_selected = set(list(features_selected_GB['feature']) + list(features_selected_RF['feature']))

# <font color='#E8800A'>Ballancing Dataset</font> <a class="anchor" id="eleventh-bullet"></a>
[Back to TOC](#toc)

## Resample to balance the dataset with SMOTE-NC

In [43]:
features_selected_1 = ['Adults','ArrivalDateDayOfMonth','ArrivalDateWeekNumber','BookingChanges','Children','DaysInWaitingList','LeadTime',\
'PreviousBookingsNotCanceled','PreviousCancellations','RequiredCarParkingSpaces','StaysInWeekNights','StaysInWeekendNights',\
'TotalOfSpecialRequests','x0_AUT','x0_BEL','x0_BRA','x0_CHE','x0_CHN','x0_DEU','x0_ESP','x0_FRA','x0_GBR','x0_ITA','x0_PRT',\
'x0_USA','x1_Transient','x1_Transient-Party','x2_Non Refund     ','x3_Direct','x3_TA/TO','x4_Corporate','x4_Direct',\
'x4_Groups','x4_Offline TA/TO','x4_Online TA','x5_HB       ','x5_SC       ','x6_1','x7_D               ','x7_E               ','x7_F               '] 

In [ ]:
# Set the range of categorical feature
cat_indices1 = list(range(13,X_train.shape[1]))
#apply smotenc for the dataset
sm = SMOTENC(random_state=5, categorical_features=cat_indices1, k_neighbors=14)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
# print the original and resampled dataset
print('Original dataset samples per class {}'.format(Counter(y_train['IsCanceled'])))
print('Resampled dataset samples per class {}'.format(Counter(y_train_res['IsCanceled'])))

### 1) Apply the Random Forest Classifier with ballanced dataset

In [ ]:
#Random Forest Classifier
RF = RandomForestClassifier()
RF.fit(X_train_res[features_selected_1], y_train_res)
y_pred_RF = RF.predict(X_val[features_selected_1])
f1_score_RF = f1_score(y_val, y_pred_RF, average='micro')
print('The f1_score of the Random Forest is:', f1_score_RF)
test_eval(RF, X_val[features_selected_1], y_val, 'Random Forest', sampling=None)

### 2) Apply the Gradient Boost Classifier with ballanced dataset

In [ ]:
GB = GradientBoostingClassifier(random_state = 5)
GB.fit(X_train_res[features_selected_1], y_train_res)
y_pred_GB = GB.predict(X_val[features_selected_1])
f1_score_GB = f1_score(y_val, y_pred_GB, average='micro')
print('The f1_score of the Gradient Boosting is:', f1_score_GB)
test_eval(GB, X_val[features_selected_1], y_val, "Gradient Boosting", sampling=None)

# <font color='#E8800A'>Tuning Models</font> <a class="anchor" id="twelveth-bullet"></a>
[Back to TOC](#toc)

## 1) Tuning Random Forest Classifier

### Maximum Depth

In [ ]:
def max_depths_tuning(train_sample, y_train_sample, x_val_sample, y_val_sample, list_max_depths):
    max_depths  = list_max_depths #, 0.1, 0.05, 0.01]
    train_results = []
    val_results = []
    for max_depth in max_depths:
        RF = RandomForestClassifier(max_depth=max_depth)
        RF.fit(train_sample, y_train_sample)
        train_pred = RF.predict(train_sample)
        f1_score_ = f1_score(y_train_sample, train_pred)
        train_results.append(f1_score_)
        y_pred = RF.predict(x_val_sample)
        f1_score_ = f1_score(y_val_sample, y_pred)
        val_results.append(f1_score_)
    from matplotlib.legend_handler import HandlerLine2D
    line1, = plt.plot(max_depths, train_results, "b", label="Train f1_score")
    line2, = plt.plot(max_depths, val_results, "r", label="val f1_score")
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel("f1 score")
    plt.xlabel("Max depths")
    plt.show()

In [ ]:
max_depths = np.linspace(13, 17, 5, endpoint=True)
max_depths

In [ ]:
max_depths_tuning(X_train_res[features_selected_1], y_train_res, X_val[features_selected_1], y_val, max_depths)

### Number of samples

In [ ]:
def nr_samples_tuning(train_sample, y_train_sample, x_val_sample, y_val_sample, list_nr_samples):
    nr_samples = list_nr_samples#np.linspace(0.1, 1.0, 10, endpoint=True)
    train_results = []
    val_results = []
    for nr_sample in nr_samples:
        RF = RandomForestClassifier(min_samples_split=nr_sample)
        RF.fit(train_sample, y_train_sample)
        train_pred = RF.predict(train_sample)
        f1_score_ = f1_score(y_train_sample, train_pred)
        train_results.append(f1_score_)
        y_pred = RF.predict(x_val_sample)
        f1_score_ = f1_score(y_val_sample, y_pred)
        val_results.append(f1_score_)
    from matplotlib.legend_handler import HandlerLine2D
    line1, = plt.plot(nr_samples, train_results, "b", label="Train f1_score")
    line2, = plt.plot(nr_samples, val_results, "r", label="val f1_score")
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel("f1 score")
    plt.xlabel("nr_samples")
    plt.show()

In [ ]:
mim= np.linspace(0.1, 0.6, 8, endpoint=True)
mim

In [ ]:
nr_samples_tuning(X_train_res[features_selected_1], y_train_res, X_val[features_selected_1], y_val, mim)

### Number of estimators

In [ ]:
def n_estimators_tuning(train_sample, y_train_sample, x_val_sample, y_val_sample, list_n_estimators):
    n_estimators = list_n_estimators
    train_results = []
    val_results = []
    for n_estimator in n_estimators:
        RF = RandomForestClassifier(n_estimators=n_estimator)
        RF.fit(train_sample, y_train_sample)
        train_pred = RF.predict(train_sample)
        f1_score_ = f1_score(y_train_sample, train_pred)
        train_results.append(f1_score_)
        y_pred = RF.predict(x_val_sample)
        f1_score_ = f1_score(y_val_sample, y_pred)
        val_results.append(f1_score_)
    from matplotlib.legend_handler import HandlerLine2D
    line1, = plt.plot(n_estimators, train_results, "b", label="Train f1_score")
    line2, = plt.plot(n_estimators, val_results, "r", label="val f1_score")
    plt.legend(handler_map={line1: HandlerLine2D(numpoints=2)})
    plt.ylabel("f1 score")
    plt.xlabel("n_estimators")
    plt.show()

In [87]:
n_estimators=list(range(1,42))

In [88]:
n_estimators_tuning(X_train_res[features_selected_1], y_train_res, X_val[features_selected_1], y_val, n_estimators)

KeyboardInterrupt: 

### Best parameters combination Random Search with Cross Validation

In [ ]:
#Random Hyperparameter Grid
#n_estimators = list(range(1,50))
#max_features = ['sqrt', 'log2']
max_depth = np.linspace(13, 17, 5, endpoint=True)
#min_samples_split = np.linspace(0.1, 0.6, 8, endpoint=True)
#min_samples_leaf = [1, 2, 4]
bootstrap = [True,False]
#max_samples = [0.2, 0.4, 0.6, 0.8, 1.0]
random_grid = {'n_estimators': n_estimators,
               #'max_features': max_features,
               'max_depth': max_depth,
               #'min_samples_split': min_samples_split,
               #'min_samples_leaf': min_samples_leaf,
               #'max_samples': max_samples,
               'bootstrap': bootstrap}
RF = RandomForestClassifier()
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
RF_best = RandomizedSearchCV(estimator = RF, param_distributions = random_grid,\
                        n_iter = 200, cv = cv, scoring='f1_micro', verbose=2, random_state=42, n_jobs = -1)
RF_best.fit(X_train_res[features_selected_1], y_train_res)

Fitting 15 folds for each of 200 candidates, totalling 3000 fits


In [ ]:
# Print best parameter for the model
RF_best.best_estimator_

In [ ]:
y_pred_RF_best = RF_best.predict(X_train_res[features_selected_1])
f1_score_RF_best = f1_score(y_train_res, y_pred_RF_best, average='micro')
print('The f1_score of the Best Random Forest is:', f1_score_RF_best)
test_eval(RF_best, X_val[features_selected_1], y_val, 'Random Forest', 'SMOTENC')

# <font color='#E8800A'>Testing the Model</font> <a class="anchor" id="fourteenth-bullet"></a>
[Back to TOC](#toc)

## Processing the test data

In [ ]:
# Transform X_val data by applying the scale obtained in the first command
min_max_X_test = min_max.transform(X_test[integer_variables_1])

In [ ]:
# Update the dataframe with the scaled features
X_test[integer_variables_1] = min_max_X_test

## Evaluating the test data

In [ ]:
test_eval(RF, X_test[features_selected_1], y_test, 'Random Forest', sampling=None)

# <font color='#E8800A'>Lift Curves</font> <a class="anchor" id="fifteenth-bullet"></a>
[Back to TOC](#toc)

# Plot lift curves Random forest

In [ ]:
# Plot lift curves
predicted_probas = RF.predict_proba(X_test[features_selected_1])

# figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(17, 9))
plot_cumulative_gain(y_test, predicted_probas, ax=ax1)
plot_lift_curve(y_test, predicted_probas, ax=ax2)  # Lift is a measure of the effectiveness of a predictive model
                                                   # calculated as the ratio between the results obtained with and 
                                                   # without the predictive model.
# properties
ax2.set_xlim(0.05, 1)
ax2.set_xticks([0.05, 0.2, 0.4, 0.6, 0.8, 1.0])
ax2.set_xticklabels([0.05, 0.2, 0.4, 0.6, 0.8, 1.0])
for ax in (ax1, ax2):
    ax.axes.lines[0].remove()
    ax.get_legend().remove()
    ax.legend()

plt.show()